In [9]:
import os, json, re
import torch
from data import RecDataModule
import pytorch_lightning as pl

PATH_DATA = "/home/ubuntu/datasets/segment_car_plate/data/"


In [10]:
with open(os.path.join(PATH_DATA, "train_recognition.json")) as fp:
    obj = json.load(fp)
fname2text = {os.path.basename(rec["file"]):rec["text"] for rec in obj}

assert len(obj) == len(fname2text)

config = []

for idx, fname in enumerate(
    os.listdir(os.path.join(PATH_DATA,"train"))
):
    if re.fullmatch(
        pattern=r"\d+\.box\.\d+\.jpg",
        string=fname
    ):
        config.append(
            (fname, fname2text[fname])
        )

In [12]:
dmodule = RecDataModule(data_path=PATH_DATA, config=config)
dmodule.setup()

dtrain = dmodule.train_dataloader()
dval = dmodule.val_dataloader()
dtest = dmodule.test_dataloader()

In [ ]:
class Model(pl.LightningModule):
    pass